# Data frame

`Microsoft.Data.Analysis`を使えばデータフレームが扱えるようになる。

In [18]:
#r "nuget:Microsoft.Data.Analysis,0.1.0"
using Microsoft.Data.Analysis;

Installing package Microsoft.Data.Analysis, version 0.1.0.done!

Successfully added reference to package Microsoft.Data.Analysis, version

## データフレームの作成

`PrivitiveDataFrameColumn<T>`で型を指定してデータフレームの列を生成できる。第2引数に長さを指定するとnullで埋められる。

In [65]:
// DateTime型の列 "DateTimes" を生成。長さ指定しなければ行数0で列だけが出来る。
var dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes");

// int型の列 "Ints" を長さ3で生成。行はnullで埋められる。
var ints = new PrimitiveDataFrameColumn<int>("Ints", 3);

// string型の列 "Strings" を長さ3で生成。行はnullで埋められる。
var strings = new StringDataFrameColumn("Strings", 3); 

In [66]:
// dateTimesに2つ値を入れる
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));

In [67]:
// 行数が合わない（dateTimesが2つしかない）ので例外が発生
DataFrame df = new DataFrame(new List<DataFrameColumn> { dateTimes, ints, strings });

Unhandled Exception: Column lengths are mismatched (Parameter 'column')

In [68]:
// 行数を合わせたらDataFrameを生成出来る。
dateTimes.Append(DateTime.Parse("2019/01/01"));

DataFrame df = new DataFrame(new List<DataFrameColumn> { dateTimes, ints, strings});

In [69]:
df

0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-01 00:00:00Z,<null>,<null>


## データフレームへの値の代入

dfの行,列指定や列のインデックス指定で直接変更出来る。

In [92]:
df[0, 1] = 10; // 0行1列に10を代入
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,125,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


In [93]:
ints[1] = 100; // ints列の行1に100を代入
strings[1] = "Foo!"; // strings列の行1に"Foo!"を代入
df

0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


In [94]:
// 型が合わない場合は例外
//ints[1] = "this will throw because I am a string";  

// intsのデータ型を確認
ints.DataType

System.Int32

ints列全体に5を足す（nullの列はそのまま）

In [96]:
df["Ints"].Add(5, inPlace: true);
df

0,2019-01-01 00:00:00Z,20,<null>
1,2019-01-01 00:00:00Z,110,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


列に対してオペレータで演算することが出来る。

In [99]:
df["Ints"] = (ints / 5) * 100;
df

0,2019-01-01 00:00:00Z,400,<null>
1,2019-01-01 00:00:00Z,2200,Foo!
2,2019-01-01 00:00:00Z,<null>,<null>


FillNullsで値を指定すると、null値のセルが指定した値で埋められる。

In [102]:
df["Ints"].FillNulls(-1, inPlace: true);
df["Strings"].FillNulls("Bar", inPlace: true);
df

0,2019-01-01 00:00:00Z,400,Bar
1,2019-01-01 00:00:00Z,2200,Foo!
2,2019-01-01 00:00:00Z,-1,Bar


## データフレームへの操作

1行目のデータを取り出してみる

In [104]:
IList<object> row0 = df[0];
row0

index Date Day DayOfWeek DayOfYear Hour Kind Millisecond Minute Month Second Ticks TimeOfDay Year 0 2019-01-01 00:00:00Z 1 { System.DayOfWeek: value__: 2 } 1 0 { System.DateTimeKind: value__: 0 } 0 0 1 0 636818976000000000 { System.TimeSpan: Ticks: 0, Days: 0, Hours: 0, Milliseconds: 0, Minutes: 0, Seconds: 0, TotalDays: 0, TotalHours: 0, TotalMilliseconds: 0, TotalMinutes: 0, TotalSeconds: 0 } 2019 1 400 2 Bar

`Filter`でFoo!以外の行をフィルタリング

In [109]:
DataFrame filtered = df.Filter(strings.ElementwiseNotEquals("Foo!"));
filtered

0,2019-01-01 00:00:00Z,400,Bar
1,2019-01-01 00:00:00Z,-1,Bar


`Sort`でInts列の昇順ソート

In [110]:
DataFrame sorted = df.Sort("Ints", ascending: true);
sorted

0,2019-01-01 00:00:00Z,-1,Bar
1,2019-01-01 00:00:00Z,400,Bar
2,2019-01-01 00:00:00Z,2200,Foo!


In [ ]:
Group

In [112]:
GroupBy groupBy = df.GroupBy("DateTimes");
// Count of values in each group
DataFrame grouped = groupBy.Count();
grouped

0,2019-01-01 00:00:00Z,3,3
